# LAB 4 : Data cleansing

| Member Name                         | Student ID           |
|----------------------------------------|-----------------------|
|Kanyaluck    Chimchome            |    64070501003   |
|Boonyarit     Samran                 |    64070501028   |  	
|Warisara      Patib                     |   64070501044    |  	
|Chanidapa   Chanama                |  64070501090    |	
|Nontawat      Kunlayawuttipong  |  64070501093   	|

## importing libraries and content files

In [364]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [365]:
cd_card = pd.read_csv("credit_card_transaction/credit_card_info.csv")
cd_tst = pd.read_csv("credit_card_transaction/credit_transaction.csv")
cd_user = pd.read_csv("credit_card_transaction/credit_users.csv")
mc_code = pd.read_csv("credit_card_transaction/mcc_codes.csv")

## viewing datatype and missing data observation

### credit card

In [366]:
cd_card.head()

,Unnamed: 0,user_id,card_index,card_brand,card_type,card_number,expires,cvv,has_chip,cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web
0,0,4e98b4b9-f30f-4117-bf0b-49c6b550aba6,0,Visa,Debit,1babcd4449ec29a14074260879c13049,12/2022,623,YES,2,$24295,09/2002,2008,No
1,1,4e98b4b9-f30f-4117-bf0b-49c6b550aba6,1,Visa,Debit,4a3fd33483704236f3af0d3e5a602f4f,12/2020,393,YES,2,$21968,04/2014,2014,No
2,2,4e98b4b9-f30f-4117-bf0b-49c6b550aba6,2,Visa,Debit,ea724c282ec208a9c826c2c6bc33b9c7,02/2024,719,YES,2,$46414,07/2003,2004,No
3,3,4e98b4b9-f30f-4117-bf0b-49c6b550aba6,3,Visa,Credit,e24a12553d70b3b18e2d4b3162d82e13,08/2024,693,NO,1,$12400,01/2003,2012,No
4,4,4e98b4b9-f30f-4117-bf0b-49c6b550aba6,4,Mastercard,Debit (Prepaid),65f39625b0df0d5f38860ab79ed2e713,03/2009,75,YES,1,$28,09/2008,2009,No


In [367]:
cd_card.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 732 entries, 0 to 731
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Unnamed: 0             732 non-null    int64 
 1   user_id                732 non-null    object
 2   card_index             732 non-null    int64 
 3   card_brand             732 non-null    object
 4   card_type              732 non-null    object
 5   card_number            732 non-null    object
 6   expires                732 non-null    object
 7   cvv                    732 non-null    int64 
 8   has_chip               732 non-null    object
 9   cards_issued           732 non-null    int64 
 10  credit_limit           732 non-null    object
 11  acct_open_date         732 non-null    object
 12  year_pin_last_changed  732 non-null    int64 
 13  card_on_dark_web       732 non-null    object
dtypes: int64(5), object(9)
memory usage: 471.1 KB


### credit transaction

In [368]:
cd_tst.head()

,Unnamed: 0,transaction_datetime,user_id,card,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors,is_fraud
0,0,2002-09-01 06:21:15,4e98b4b9-f30f-4117-bf0b-49c6b550aba6,0,134.09,Swipe Transaction,aec1f6b6-0713-480c-ba05-2bf31892f39c,La Verne,CA,91750.0,5300,NaN,No
1,1,2002-09-01 06:42:17,4e98b4b9-f30f-4117-bf0b-49c6b550aba6,0,38.48,Swipe Transaction,9f645ffc-fa0a-44ec-8e60-7533bccf63d9,Monterey Park,CA,91754.0,5411,NaN,No
2,2,2002-09-02 06:22:57,4e98b4b9-f30f-4117-bf0b-49c6b550aba6,0,120.34,Swipe Transaction,9f645ffc-fa0a-44ec-8e60-7533bccf63d9,Monterey Park,CA,91754.0,5411,NaN,No
3,3,2002-09-02 17:45:54,4e98b4b9-f30f-4117-bf0b-49c6b550aba6,0,128.95,Swipe Transaction,fc611f1f-7adf-42b3-b741-1749bb07eacf,Monterey Park,CA,91754.0,5651,NaN,No
4,4,2002-09-03 06:23:01,4e98b4b9-f30f-4117-bf0b-49c6b550aba6,0,104.71,Swipe Transaction,370f2dcc-3168-4b18-a6c6-8e612709695a,La Verne,CA,91750.0,5912,NaN,No


In [369]:
cd_tst.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2969557 entries, 0 to 2969556
Data columns (total 13 columns):
 #   Column                Dtype  
---  ------                -----  
 0   Unnamed: 0            int64  
 1   transaction_datetime  object 
 2   user_id               object 
 3   card                  int64  
 4   amount                float64
 5   use_chip              object 
 6   merchant_id           object 
 7   merchant_city         object 
 8   merchant_state        object 
 9   zip                   float64
 10  mcc                   int64  
 11  errors                object 
 12  is_fraud              object 
dtypes: float64(2), int64(3), object(8)
memory usage: 294.5+ MB


In [370]:
cd_tst.isna().sum()

Unnamed: 0                    0
transaction_datetime          0
user_id                       0
card                          0
amount                        0
use_chip                      0
merchant_id                   0
merchant_city                 0
merchant_state           377166
zip                      398187
mcc                           0
errors                  2924122
is_fraud                      0
dtype: int64

### credit users

In [371]:
cd_user.head()

,Unnamed: 0,user_id,person,date_of_birth,retirement_age,gender,address,apartment,city,state,zipcode,per_capita_income_zipcode,yearly_income_person,total_debt,fico_score,num_credit_cards
0,0,4e98b4b9-f30f-4117-bf0b-49c6b550aba6,Hazel Robinson,1966/11/24,66,Female,462 Rose Lane,NaN,La Verne,CA,91750,$29278,$59696,$127613,787,5
1,1,8e283e3b-fb65-4d2a-9c7e-b515c94d8402,Sasha Sadr,1966/12/25,68,Female,3606 Federal Boulevard,NaN,Little Neck,NY,11363,$37891,$77254,$191349,701,5
2,2,8c9b9223-c41c-4999-9cff-f72b9c4bcf7c,Saanvi Lee,1938/11/25,67,Female,766 Third Drive,NaN,West Covina,CA,91792,$22681,$33483,$196,698,5
3,3,acab43d2-f982-47a1-9b4f-d018d069ae16,Everlee Clark,1957/1/16,63,Female,3 Madison Street,NaN,New York,NY,10069,$163145,$249925,$202328,722,4
4,4,80884e14-f347-48c3-8848-768e4c33d693,Kyle Peterson,1976/9/3,70,Male,9620 Valley Stream Drive,NaN,San Francisco,CA,94117,$53797,$109687,$183855,675,1


In [372]:
cd_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Unnamed: 0                 250 non-null    int64  
 1   user_id                    250 non-null    object 
 2   person                     250 non-null    object 
 3   date_of_birth              250 non-null    object 
 4   retirement_age             250 non-null    int64  
 5   gender                     250 non-null    object 
 6   address                    250 non-null    object 
 7   apartment                  69 non-null     float64
 8   city                       250 non-null    object 
 9   state                      250 non-null    object 
 10  zipcode                    250 non-null    int64  
 11  per_capita_income_zipcode  250 non-null    object 
 12  yearly_income_person       250 non-null    object 
 13  total_debt                 250 non-null    object 

In [373]:
cd_user.isna().sum()

Unnamed: 0                     0
user_id                        0
person                         0
date_of_birth                  0
retirement_age                 0
gender                         0
address                        0
apartment                    181
city                           0
state                          0
zipcode                        0
per_capita_income_zipcode      0
yearly_income_person           0
total_debt                     0
fico_score                     0
num_credit_cards               0
dtype: int64

### mcc_codes

In [374]:
mc_code.head()

,mcc,edited_description,combined_description,usda_description,irs_description,irs_reportable
0,742,Veterinary Services,Veterinary Services,Veterinary Services,Veterinary Services,Yes
1,763,Agricultural Co-operatives,Agricultural Co-operatives,Agricultural Co-operatives,Agricultural Cooperative,Yes
2,780,"Horticultural Services, Landscaping Services","Horticultural Services, Landscaping Services",Horticultural Services,Landscaping Services,Yes
3,1520,General Contractors-Residential and Commercial,General Contractors-Residential and Commercial,General Contractors-Residential and Commercial,General Contractors,Yes
4,1711,Air Conditioning Contractors – Sales and Insta...,Air Conditioning Contractors – Sales and Insta...,Air Conditioning Contractors – Sales and Insta...,"Heating, Plumbing, A/C",Yes


In [375]:
mc_code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 981 entries, 0 to 980
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   mcc                   981 non-null    int64 
 1   edited_description    981 non-null    object
 2   combined_description  973 non-null    object
 3   usda_description      706 non-null    object
 4   irs_description       963 non-null    object
 5   irs_reportable        963 non-null    object
dtypes: int64(1), object(5)
memory usage: 46.1+ KB


In [376]:
mc_code.isna().sum()

mcc                       0
edited_description        0
combined_description      8
usda_description        275
irs_description          18
irs_reportable           18
dtype: int64

## Coding 

In [377]:
user_prof = pd.DataFrame()

In [378]:
user_prof["user_id"] = cd_user.user_id

1. Ages in year

In [379]:
# get referrence date from max transaction date
ref_date = max(pd.to_datetime(cd_tst.transaction_datetime))
ref_date

Timestamp('2020-02-28 23:49:03')

In [380]:
user_prof["current_age"] = (ref_date - pd.to_datetime(cd_user.date_of_birth)).dt.days // 365

2. Number of active and expired cards

In [381]:
from pandas.tseries.offsets import MonthEnd

# pd.to_datetime(cd_card["expires"]).dt.strftime('%Y-%m')
tempDate = pd.to_datetime(cd_card["expires"], format="%m/%Y")
tempExp = pd.DataFrame((tempDate - ref_date).dt.days)

tempExp["user_id"] = cd_card["user_id"]
tempExp["expired_cards"] = tempExp.expires < 0
test = tempExp.copy()
tempExp = tempExp.groupby(["user_id","expired_cards"])['expires'].agg(['count']).unstack(fill_value=0)

user_prof = pd.merge(user_prof, tempExp, on="user_id", how='left')
user_prof.columns = ['user_id', 'current_age', 'active_cards', 'expired_cards']


C:\Users\Parewa\AppData\Local\Temp\ipykernel_6240\244032723.py:12: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  user_prof = pd.merge(user_prof, tempExp, on="user_id", how='left')


In [382]:
user_prof


,user_id,current_age,active_cards,expired_cards
0,4e98b4b9-f30f-4117-bf0b-49c6b550aba6,53,4,1
1,8e283e3b-fb65-4d2a-9c7e-b515c94d8402,53,4,1
2,8c9b9223-c41c-4999-9cff-f72b9c4bcf7c,81,4,1
3,acab43d2-f982-47a1-9b4f-d018d069ae16,63,3,1
4,80884e14-f347-48c3-8848-768e4c33d693,43,1,0
...,...,...,...,...
245,2c4e57b3-7314-48fc-8e33-3325fc13c990,18,0,1
246,9b9ce53a-d20d-499c-a1a3-4e3eb9b56917,50,2,0
247,91c1a49f-ed8d-4b31-943f-d1300b2560b6,68,2,0
248,b4fa8930-c642-49ca-be86-7b031dd46067,26,0,1


3. Total credit limit of active cards

4. Time since first card openned (acct_open_date) in months

#### Create Last Calendar Year Dataframe

In [383]:
tst_lcy = cd_tst.copy()
tst_lcy['transaction_datetime'] = pd.to_datetime(tst_lcy['transaction_datetime'])

# copy only transaction data in 2019 (Last year transaction calendar)
tst_lcy['Year'] = tst_lcy['transaction_datetime'].dt.year
tst_lcy = tst_lcy.loc[tst_lcy['Year']== 2019]
tst_lcy.drop(columns='Year', inplace=True)
tst_lcy.to_string(index=False)

# create last calendar year transaction data frame that have already drop the error transaction
tst_lcy_dperr = tst_lcy[tst_lcy.errors.notna()]
tst_lcy


,Unnamed: 0,transaction_datetime,user_id,card,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors,is_fraud
4776,4776,2019-01-05 06:02:56,4e98b4b9-f30f-4117-bf0b-49c6b550aba6,0,120.30,Chip Transaction,e12b0ab7-167a-4dc3-81ed-4a3fa74db901,Princeton,MA,1541.0,5411,Insufficient Balance,No
4777,4777,2019-01-06 20:10:10,4e98b4b9-f30f-4117-bf0b-49c6b550aba6,0,68.43,Chip Transaction,ea16e3d6-9543-410d-ac25-c741401ebec2,North Grafton,MA,1536.0,5812,NaN,No
4778,4778,2019-01-09 10:18:03,4e98b4b9-f30f-4117-bf0b-49c6b550aba6,0,59.17,Chip Transaction,fa234243-ab93-4f26-beaa-15de51b3cb90,North Grafton,MA,1536.0,7538,NaN,No
4779,4779,2019-01-11 13:13:45,4e98b4b9-f30f-4117-bf0b-49c6b550aba6,0,16.07,Chip Transaction,e12b0ab7-167a-4dc3-81ed-4a3fa74db901,Princeton,MA,1541.0,5411,NaN,No
4780,4780,2019-01-12 05:54:35,4e98b4b9-f30f-4117-bf0b-49c6b550aba6,0,132.82,Chip Transaction,60e4760a-eb92-45d9-a6ad-979c221c36eb,Sturbridge,MA,1566.0,5300,NaN,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2969276,2969276,2019-12-31 07:21:36,3314e0d1-ce54-4f22-a04e-31bb78492bda,0,194.82,Swipe Transaction,5a51af6f-5bee-4928-857b-b78f560ad902,Oklahoma City,OK,73159.0,5411,NaN,No
2969277,2969277,2019-12-31 07:56:49,3314e0d1-ce54-4f22-a04e-31bb78492bda,0,229.65,Swipe Transaction,86ebca55-9947-4b5b-bcb8-787e90e7ed16,Oklahoma City,OK,73151.0,5912,NaN,No
2969278,2969278,2019-12-31 09:43:44,3314e0d1-ce54-4f22-a04e-31bb78492bda,0,7.02,Swipe Transaction,ccc05ea9-8bf2-45e7-8b47-be7b84286970,Duncan,OK,73533.0,5942,NaN,No
2969279,2969279,2019-12-31 10:16:39,3314e0d1-ce54-4f22-a04e-31bb78492bda,0,64.72,Online Transaction,1f8846de-211c-4266-891f-d4dfb09eee48,ONLINE,NaN,NaN,5311,NaN,No


20. Last calendar year transaction count

In [384]:
temp_trans = tst_lcy[['user_id']].copy()
temp_trans = temp_trans.groupby(['user_id']).agg({'user_id':'count'})
temp_trans = temp_trans.rename(columns={'user_id':'last_year_tx_count'})
temp_trans.reset_index()
user_prof = pd.merge(user_prof, temp_trans, on="user_id", how='left').fillna(0)
user_prof

,user_id,current_age,active_cards,expired_cards,last_year_tx_count
0,4e98b4b9-f30f-4117-bf0b-49c6b550aba6,53,4,1,1164.0
1,8e283e3b-fb65-4d2a-9c7e-b515c94d8402,53,4,1,566.0
2,8c9b9223-c41c-4999-9cff-f72b9c4bcf7c,81,4,1,2573.0
3,acab43d2-f982-47a1-9b4f-d018d069ae16,63,3,1,860.0
4,80884e14-f347-48c3-8848-768e4c33d693,43,1,0,921.0
...,...,...,...,...,...
245,2c4e57b3-7314-48fc-8e33-3325fc13c990,18,0,1,0.0
246,9b9ce53a-d20d-499c-a1a3-4e3eb9b56917,50,2,0,408.0
247,91c1a49f-ed8d-4b31-943f-d1300b2560b6,68,2,0,1238.0
248,b4fa8930-c642-49ca-be86-7b031dd46067,26,0,1,341.0


21. Last calendar year amount used

In [385]:
# use dataframe that have already drop the error transaction to find Last calendar amount used
temp_amount = tst_lcy_dperr[['user_id','amount']].copy()
temp_amount = temp_amount.groupby(['user_id']).agg({'amount':'sum'})
temp_amount = temp_amount.rename(columns={'amount':'last_year_amount_used'})

# merge 2 dataframe together by user_id
user_prof = pd.merge(user_prof, temp_amount, on="user_id", how='left').fillna(0)
user_prof

,user_id,current_age,active_cards,expired_cards,last_year_tx_count,last_year_amount_used
0,4e98b4b9-f30f-4117-bf0b-49c6b550aba6,53,4,1,1164.0,4005.05
1,8e283e3b-fb65-4d2a-9c7e-b515c94d8402,53,4,1,566.0,1396.14
2,8c9b9223-c41c-4999-9cff-f72b9c4bcf7c,81,4,1,2573.0,1702.24
3,acab43d2-f982-47a1-9b4f-d018d069ae16,63,3,1,860.0,3282.08
4,80884e14-f347-48c3-8848-768e4c33d693,43,1,0,921.0,921.23
...,...,...,...,...,...,...
245,2c4e57b3-7314-48fc-8e33-3325fc13c990,18,0,1,0.0,0.00
246,9b9ce53a-d20d-499c-a1a3-4e3eb9b56917,50,2,0,408.0,115.83
247,91c1a49f-ed8d-4b31-943f-d1300b2560b6,68,2,0,1238.0,681.70
248,b4fa8930-c642-49ca-be86-7b031dd46067,26,0,1,341.0,23.61


22. Last calendar year median amount per transaction

In [386]:
# use dataframe that have already drop the error transaction to find Last calendar amount used
temp_amount_md = tst_lcy_dperr[['user_id','amount']].copy()
temp_amount_md = temp_amount_md.groupby(['user_id']).agg({'amount':'median'})
temp_amount_md = temp_amount_md.rename(columns={'amount':'last_year_median_amount_per_tx'})

# merge 2 dataframe together by user_id
user_prof = pd.merge(user_prof, temp_amount_md, on="user_id", how='left').fillna(0)
user_prof

,user_id,current_age,active_cards,expired_cards,last_year_tx_count,last_year_amount_used,last_year_median_amount_per_tx
0,4e98b4b9-f30f-4117-bf0b-49c6b550aba6,53,4,1,1164.0,4005.05,119.865
1,8e283e3b-fb65-4d2a-9c7e-b515c94d8402,53,4,1,566.0,1396.14,98.540
2,8c9b9223-c41c-4999-9cff-f72b9c4bcf7c,81,4,1,2573.0,1702.24,44.445
3,acab43d2-f982-47a1-9b4f-d018d069ae16,63,3,1,860.0,3282.08,111.390
4,80884e14-f347-48c3-8848-768e4c33d693,43,1,0,921.0,921.23,98.100
...,...,...,...,...,...,...,...
245,2c4e57b3-7314-48fc-8e33-3325fc13c990,18,0,1,0.0,0.00,0.000
246,9b9ce53a-d20d-499c-a1a3-4e3eb9b56917,50,2,0,408.0,115.83,5.955
247,91c1a49f-ed8d-4b31-943f-d1300b2560b6,68,2,0,1238.0,681.70,49.310
248,b4fa8930-c642-49ca-be86-7b031dd46067,26,0,1,341.0,23.61,11.805


23. Last calendar year number of fraud transaction

In [387]:
temp_fraud = tst_lcy[['user_id','is_fraud']].copy()
temp_fraud = temp_fraud[temp_fraud.is_fraud == 'Yes']
temp_fraud = temp_fraud.groupby(['user_id']).agg({'is_fraud':'count'})
temp_fraud = temp_fraud.rename(columns={'is_fraud':'last_year_fraud_tx_count'})

user_prof = pd.merge(user_prof, temp_fraud, on="user_id", how='left').fillna(0)
user_prof

,user_id,current_age,active_cards,expired_cards,last_year_tx_count,last_year_amount_used,last_year_median_amount_per_tx,last_year_fraud_tx_count
0,4e98b4b9-f30f-4117-bf0b-49c6b550aba6,53,4,1,1164.0,4005.05,119.865,0.0
1,8e283e3b-fb65-4d2a-9c7e-b515c94d8402,53,4,1,566.0,1396.14,98.540,10.0
2,8c9b9223-c41c-4999-9cff-f72b9c4bcf7c,81,4,1,2573.0,1702.24,44.445,0.0
3,acab43d2-f982-47a1-9b4f-d018d069ae16,63,3,1,860.0,3282.08,111.390,0.0
4,80884e14-f347-48c3-8848-768e4c33d693,43,1,0,921.0,921.23,98.100,0.0
...,...,...,...,...,...,...,...,...
245,2c4e57b3-7314-48fc-8e33-3325fc13c990,18,0,1,0.0,0.00,0.000,0.0
246,9b9ce53a-d20d-499c-a1a3-4e3eb9b56917,50,2,0,408.0,115.83,5.955,0.0
247,91c1a49f-ed8d-4b31-943f-d1300b2560b6,68,2,0,1238.0,681.70,49.310,0.0
248,b4fa8930-c642-49ca-be86-7b031dd46067,26,0,1,341.0,23.61,11.805,0.0


24. Last calendar year number of refunded transaction (negative amount)

In [388]:
temp_refunded = tst_lcy[['user_id','amount']].copy()
temp_refunded = temp_refunded[temp_refunded.amount < 0]
temp_refunded = temp_refunded.groupby(['user_id']).agg({'amount':'count'})
temp_refunded = temp_refunded.rename(columns={'amount':'last_year_refunded_tx_count'})
user_prof = pd.merge(user_prof, temp_refunded, on="user_id", how='left').fillna(0)
user_prof

,user_id,current_age,active_cards,expired_cards,last_year_tx_count,last_year_amount_used,last_year_median_amount_per_tx,last_year_fraud_tx_count,last_year_refunded_tx_count
0,4e98b4b9-f30f-4117-bf0b-49c6b550aba6,53,4,1,1164.0,4005.05,119.865,0.0,35.0
1,8e283e3b-fb65-4d2a-9c7e-b515c94d8402,53,4,1,566.0,1396.14,98.540,10.0,31.0
2,8c9b9223-c41c-4999-9cff-f72b9c4bcf7c,81,4,1,2573.0,1702.24,44.445,0.0,31.0
3,acab43d2-f982-47a1-9b4f-d018d069ae16,63,3,1,860.0,3282.08,111.390,0.0,41.0
4,80884e14-f347-48c3-8848-768e4c33d693,43,1,0,921.0,921.23,98.100,0.0,11.0
...,...,...,...,...,...,...,...,...,...
245,2c4e57b3-7314-48fc-8e33-3325fc13c990,18,0,1,0.0,0.00,0.000,0.0,0.0
246,9b9ce53a-d20d-499c-a1a3-4e3eb9b56917,50,2,0,408.0,115.83,5.955,0.0,23.0
247,91c1a49f-ed8d-4b31-943f-d1300b2560b6,68,2,0,1238.0,681.70,49.310,0.0,29.0
248,b4fa8930-c642-49ca-be86-7b031dd46067,26,0,1,341.0,23.61,11.805,0.0,15.0


25. Last calendar year fraud transaction percentage

In [389]:
user_prof['last_year_fraud_tx_percentage'] = (user_prof['last_year_fraud_tx_count'] / user_prof['last_year_tx_count'])
user_prof = user_prof.fillna(0)
user_prof

,user_id,current_age,active_cards,expired_cards,last_year_tx_count,last_year_amount_used,last_year_median_amount_per_tx,last_year_fraud_tx_count,last_year_refunded_tx_count,last_year_fraud_tx_percentage
0,4e98b4b9-f30f-4117-bf0b-49c6b550aba6,53,4,1,1164.0,4005.05,119.865,0.0,35.0,0.000000
1,8e283e3b-fb65-4d2a-9c7e-b515c94d8402,53,4,1,566.0,1396.14,98.540,10.0,31.0,0.017668
2,8c9b9223-c41c-4999-9cff-f72b9c4bcf7c,81,4,1,2573.0,1702.24,44.445,0.0,31.0,0.000000
3,acab43d2-f982-47a1-9b4f-d018d069ae16,63,3,1,860.0,3282.08,111.390,0.0,41.0,0.000000
4,80884e14-f347-48c3-8848-768e4c33d693,43,1,0,921.0,921.23,98.100,0.0,11.0,0.000000
...,...,...,...,...,...,...,...,...,...,...
245,2c4e57b3-7314-48fc-8e33-3325fc13c990,18,0,1,0.0,0.00,0.000,0.0,0.0,0.000000
246,9b9ce53a-d20d-499c-a1a3-4e3eb9b56917,50,2,0,408.0,115.83,5.955,0.0,23.0,0.000000
247,91c1a49f-ed8d-4b31-943f-d1300b2560b6,68,2,0,1238.0,681.70,49.310,0.0,29.0,0.000000
248,b4fa8930-c642-49ca-be86-7b031dd46067,26,0,1,341.0,23.61,11.805,0.0,15.0,0.000000


26. Last calendar year median duration between transaction in hours

In [390]:
temp_duration = tst_lcy[['user_id','transaction_datetime']].copy()
temp_duration = temp_duration.sort_values(['user_id','transaction_datetime'],ascending=True)
temp_duration['duration'] = temp_duration['transaction_datetime'].diff() / pd.Timedelta(hours=1)
temp_duration

# the problem is can't find the diff only the same ID
temp_duration = temp_duration.groupby(['user_id']).agg({'duration':'median'})
temp_duration = temp_duration.rename(columns={'duration':'last_year_median_time_between_tx_hours'})

user_prof = pd.merge(user_prof, temp_duration, on="user_id", how='left').fillna(0)
user_prof['last_year_median_time_between_tx_hours'] = round(user_prof['last_year_median_time_between_tx_hours'])
user_prof

,user_id,current_age,active_cards,expired_cards,last_year_tx_count,last_year_amount_used,last_year_median_amount_per_tx,last_year_fraud_tx_count,last_year_refunded_tx_count,last_year_fraud_tx_percentage,last_year_median_time_between_tx_hours
0,4e98b4b9-f30f-4117-bf0b-49c6b550aba6,53,4,1,1164.0,4005.05,119.865,0.0,35.0,0.000000,5.0
1,8e283e3b-fb65-4d2a-9c7e-b515c94d8402,53,4,1,566.0,1396.14,98.540,10.0,31.0,0.017668,10.0
2,8c9b9223-c41c-4999-9cff-f72b9c4bcf7c,81,4,1,2573.0,1702.24,44.445,0.0,31.0,0.000000,2.0
3,acab43d2-f982-47a1-9b4f-d018d069ae16,63,3,1,860.0,3282.08,111.390,0.0,41.0,0.000000,6.0
4,80884e14-f347-48c3-8848-768e4c33d693,43,1,0,921.0,921.23,98.100,0.0,11.0,0.000000,6.0
...,...,...,...,...,...,...,...,...,...,...,...
245,2c4e57b3-7314-48fc-8e33-3325fc13c990,18,0,1,0.0,0.00,0.000,0.0,0.0,0.000000,0.0
246,9b9ce53a-d20d-499c-a1a3-4e3eb9b56917,50,2,0,408.0,115.83,5.955,0.0,23.0,0.000000,15.0
247,91c1a49f-ed8d-4b31-943f-d1300b2560b6,68,2,0,1238.0,681.70,49.310,0.0,29.0,0.000000,5.0
248,b4fa8930-c642-49ca-be86-7b031dd46067,26,0,1,341.0,23.61,11.805,0.0,15.0,0.000000,6.0
